In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc3_preprocessor import Preprocessor

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


## Vanilla CatBoost Model (Default Parameters)

In [8]:
cb_clf = cb.CatBoostClassifier(
    objective='Logloss',
    random_state=42,
    thread_count=-1
)

In [9]:
cat_feature_names = pre.cat_for_encoding_
cb_clf.fit(X_train_proc, y_train, 
           cat_features=cat_feature_names,
           verbose=False)

In [10]:
test_probabilities = cb_clf.predict_proba(X_test_proc)[:, 1]

test_classes = cb_clf.predict(X_test_proc)

print(f"Accuracy: {accuracy_score(y_test, test_classes)}")
print(f"F1 Score: {f1_score(y_test, test_classes)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_probabilities)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_probabilities)}") # Use probabilities
print(f"Precision: {precision_score(y_test, test_classes)}")
print(f"Recall: {recall_score(y_test, test_classes)}")

Accuracy: 0.8137037037037037
F1 Score: 0.5204957102001907
ROC AUC Score: 0.8388279036310881
PR AUC (Average Precision): 0.6048398830102898
Precision: 0.6326767091541136
Recall: 0.4421052631578947


## CatBoost with Optuna Tuning

In [11]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [13]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)

    pruning_cb = CatBoostPruningCallback(trial, "Logloss")
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False,
        callbacks=[pruning_cb]
    )

    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [14]:
print("\n2. Starting Optuna study...")

MAX_ITERATIONS = 1000

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.HyperbandPruner( 
        min_resource=1,
        max_resource=MAX_ITERATIONS, # Tells Hyperband the max iterations
        reduction_factor=3
    )
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-06 17:28:46,930] A new study created in memory with name: no-name-c65866c3-10e4-443c-b60a-17dfcf4a7786



2. Starting Optuna study...


  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")
Best trial: 0. Best value: 0.551779:   1%|          | 1/100 [00:06<10:43,  6.50s/it]/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")


[I 2025-11-06 17:28:53,432] Trial 0 finished with value: 0.5517786561264822 and parameters: {'learning_rate': 0.021556678012661715, 'depth': 10, 'l2_leaf_reg': 0.001072370556175943, 'subsample': 0.7, 'random_strength': 1.0655225386985244e-06, 'bagging_temperature': 0.7608744448647068, 'border_count': 86, 'scale_pos_weight': 9.30150988805618}. Best is trial 0 with value: 0.5517786561264822.


Best trial: 1. Best value: 0.580141:   2%|▏         | 2/100 [00:08<06:40,  4.08s/it]/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")


...

[I 2025-11-06 17:30:19,878] Trial 93 finished with value: 0.5860400829302004 and parameters: {'learning_rate': 0.0110949072928277, 'depth': 5, 'l2_leaf_reg': 2.7487486120791496, 'subsample': 1.0, 'random_strength': 0.00011926043713787389, 'bagging_temperature': 0.8168597165371808, 'border_count': 84, 'scale_pos_weight': 2.005739706792013}. Best is trial 21 with value: 0.5998694516971279.
[I 2025-11-06 17:30:19,991] Trial 94 finished with value: 0.5295415543025251 and parameters: {'learning_rate': 0.011340214720315225, 'depth': 5, 'l2_leaf_reg': 2.4840234632834535, 'subsample': 1.0, 'random_strength': 0.00020741912329580664, 'bagging_temperature': 0.8546269498701865, 'border_count': 62, 'scale_pos_weight': 6.9204759141688434}. Best is trial 21 with value: 0.5998694516971279.
[I 2025-11-06 17:30:20,053] Trial 95 finished with value: 0.5780310224754669 and parameters: {'learning_rate': 0.01726536339925138, 'depth': 6, 'l2_leaf_reg': 1.6288890678268932, 'subsample': 1.0, 'random_strength':

Best trial: 21. Best value: 0.599869:  97%|█████████▋| 97/100 [01:33<00:00,  3.84it/s]/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")
Best trial: 21. Best value: 0.599869:  98%|█████████▊| 98/100 [01:33<00:00,  4.41it/s]/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")
Best trial: 21. Best value: 0.599869:  98%|█████████▊| 98/100 [01:33<00:00,  4.41it/s]

[I 2025-11-06 17:30:20,131] Trial 96 finished with value: 0.5781151170145478 and parameters: {'learning_rate': 0.010774718020328415, 'depth': 5, 'l2_leaf_reg': 3.0571969737001456, 'subsample': 0.9, 'random_strength': 1.586305013060817e-08, 'bagging_temperature': 0.8133392853765729, 'border_count': 69, 'scale_pos_weight': 2.1438639622019937}. Best is trial 21 with value: 0.5998694516971279.
[I 2025-11-06 17:30:20,238] Trial 97 finished with value: 0.5354969574036511 and parameters: {'learning_rate': 0.01526360582384809, 'depth': 4, 'l2_leaf_reg': 2.1389198504100584, 'subsample': 1.0, 'random_strength': 0.07300731671493839, 'bagging_temperature': 0.19691737670554688, 'border_count': 89, 'scale_pos_weight': 4.021978861822159}. Best is trial 21 with value: 0.5998694516971279.
[I 2025-11-06 17:30:20,329] Trial 98 finished with value: 0.5627270187203129 and parameters: {'learning_rate': 0.05726350650431647, 'depth': 7, 'l2_leaf_reg': 8.072185739993252, 'subsample': 0.5, 'random_strength': 0.

/var/folders/cb/0wwk4_d90ps1fw1_wg2m60340000gn/T/ipykernel_34556/1771674398.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_cb = CatBoostPruningCallback(trial, "Logloss")
Best trial: 21. Best value: 0.599869: 100%|██████████| 100/100 [01:33<00:00,  1.07it/s]

[I 2025-11-06 17:30:20,405] Trial 99 finished with value: 0.5743685687558466 and parameters: {'learning_rate': 0.01882330332910022, 'depth': 6, 'l2_leaf_reg': 5.398984888063465, 'subsample': 0.9, 'random_strength': 6.8249138560461234e-06, 'bagging_temperature': 0.8698616605332148, 'border_count': 42, 'scale_pos_weight': 2.8592269437310094}. Best is trial 21 with value: 0.5998694516971279.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.5999
  Best Hyperparameters:
    learning_rate: 0.011120804939988044
    depth: 4
    l2_leaf_reg: 0.3203896185865553
    subsample: 0.9
    random_strength: 0.03443317052185157
    bagging_temperature: 0.3175949378538905
    border_count: 59
    scale_pos_weight: 2.6612180500653415


In [19]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 1000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 50 # Keep early stopping
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.011120804939988044, 'depth': 4, 'l2_leaf_reg': 0.3203896185865553, 'subsample': 0.9, 'random_strength': 0.03443317052185157, 'bagging_temperature': 0.3175949378538905, 'border_count': 59, 'scale_pos_weight': 2.6612180500653415}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5999

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.91      0.78      0.84      4165
   Class 1.0       0.50      0.74      0.60      1235

    accuracy                           0.77      5400
   macro avg       0.71      0.76      0.72      5400
weighted avg       0.82      0.77      0.79      5400



In [20]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [17]:
# prediction.to_csv("results/catboost_hyperband_prediction.csv", index=False)

In [21]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 136

Top 10 Most Important Features:
                        feature  importance
44              liab_x_multicar   12.575262
135  recovery_feasibility_score    7.330751
55                 liab_inverse    5.605071
45         liab_x_highrisk_site    5.448074
13                    liab_prct    4.695390
54                liab_prct_log    4.604083
52              liab_prct_cubed    4.423122
58     is_multi_vehicle_unclear    4.225854
51            liab_prct_squared    3.843335
56         liab_inverse_squared    3.649937

Bottom 10 Least Important Features:
                   feature  importance
91           light_vehicle         0.0
93      is_compact_vehicle         0.0
94            high_mileage         0.0
97        frequent_claimer         0.0
99      first_time_claimer         0.0
100  very_frequent_claimer         0.0
101           large_payout         0.0
102          medium_payout         0.0
103           small_payout         0.0
